In [1]:
import phyloGraph as ph

from plotly.offline import download_plotlyjs, init_notebook_mode
init_notebook_mode(connected=True)

#vim ~/.Plotly/.Credentials

import requests
import collections
import json
import pandas as pd
import numpy as np
import re
import sys
from time import time

import wikipedia
#https://stackoverflow.com/questions/8088226/content-of-infobox-of-wikipedia

In [2]:
GEO_TIME = "geological_time.csv"
#DATA_FILE = "data/Mammalia-15040-df-fixedage.csv"
#OUT_FILE = 'data/Mammalia-15040-df-images.csv'

DATA_FILE = "data/Chordata-2499-df-fixedage.csv"
OUT_FILE = 'data/Chordata-2499-df-images.csv'

In [3]:
gt = pd.read_csv(GEO_TIME)

def trynum(n):
    try:
        return float(n)
    except:
        return None

In [4]:
df = pd.read_csv(DATA_FILE)
print(df.shape)
print(df.columns)

(10395, 11)
Index(['ancestor', 'depth', 'extinct', 'id', 'name', 'num_kids', 'phylesis',
       'x', 'y', 'Begin', 'End'],
      dtype='object')


In [5]:
this_query = df.name[0]
this_query

'Chordata'

In [6]:
start = time()

In [7]:
def parse_png_from_box(box):
    png_str = [b for b in box.split('|') if (('.png' in b) or ('.jpg' in b)) and ('image' in b)][0]
    #clean up string
    png_str = re.sub("\\n", '', png_str)
    png_str = re.sub("^(.+?)image +?= +?", '', png_str)
    png_str = re.sub("^(.+?)File:", '', png_str)
    png_str = png_str.replace(" ", "_")
    return png_str

def get_image_url(this_query, 
                  base_url = "https://en.wikipedia.org/wiki/File:",
                  fallback = "https://en.wikipedia.org/wiki/File:Wikipedia-logo-v2.svg"):
    """
    """
    try:
        # get wikipedia page
        this_page_id = wikipedia.search(this_query, results=1)[0]
    
        # fetch infobox
        wiki_url = 'http://en.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&titles={}&rvsection=0'.format(this_page_id)
        box_raw = json.loads(requests.get(wiki_url).content.decode('utf-8'))
        box = next(iter(box_raw['query']['pages'].values()))['revisions'][0]['*']
        
        png_str = parse_png_from_box(box)
        
        return base_url + png_str
    except:
        return fallback

In [8]:
get_image_url(df.name[0])

'https://en.wikipedia.org/wiki/File:image=Kryptopterus.jpg'

In [9]:
img_list = []
for i, row in df.iterrows():
    img_list.append(get_image_url(row['name']))
    if i % 50 == 0:
        print("{}::{}".format(i, int(time()-start)), end= ' ', flush=True)

0::3 50::68 100::130 150::187 200::260 250::320 300::388 350::462 400::521 450::594 500::668 550::744 600::804 650::856 700::928 750::991 800::1056 850::1109 900::1176 950::1234 1000::1278 1050::1318 1100::1352 1150::1384 1200::1418 1250::1460 1300::1496 1350::1533 1400::1588 1450::1628 1500::1676 1550::1726 1600::1771 1650::1818 1700::1855 1750::1906 1800::1956 1850::1997 1900::2041 1950::2087 2000::2135 2050::2187 2100::2229 2150::2275 2200::2317 2250::2359 2300::2409 2350::2460 2400::2508 2450::2550 2500::2605 2550::2662 2600::2716 2650::2765 2700::2820 2750::2872 2800::2916 2850::2977 2900::3036 2950::3090 3000::3140 3050::3193 3100::3250 3150::3299 3200::3343 3250::3389 3300::3438 3350::3499 3400::3538 3450::3575 3500::3612 3550::3650 3600::3684 3650::3729 3700::3769 3750::3814 3800::3855 3850::3896 3900::3941 3950::3991 4000::4061 4050::4127 4100::4198 4150::4287 4200::4384 4250::4461 4300::4550 4350::4643 4400::4721 4450::4794 4500::4846 4550::4900 4600::4969 4650::5029 4700::50

In [10]:
df['img_url'] = img_list
df.tail()

,ancestor,depth,extinct,id,name,num_kids,phylesis,x,y,Begin,End,img_url
10390,15088,9,0,15091,Polymixiiformes,0,0,-1.329644,1.238158,34.0,0.0,https://en.wikipedia.org/wiki/File:Polymixia_n...
10391,15088,9,0,15089,Lampridiformes,0,0,-1.387254,0.381895,0.0,0.0,https://en.wikipedia.org/wiki/File:Lophotus_la...
10392,14843,5,2,14924,Acanthodii,0,0,-1.555681,0.007475,188.0,0.0,https://en.wikipedia.org/wiki/File:image_=_Aca...
10393,14843,5,0,14925,Chondrichthyes,0,0,-2.013908,0.478732,188.0,0.0,https://en.wikipedia.org/wiki/File:White_shark...
10394,14843,5,2,14926,Placodermi,0,0,-1.105787,0.408763,188.0,0.0,https://en.wikipedia.org/wiki/File:Bothriolepi...


In [11]:
print(df.shape)
print(df.groupby('img_url').count().sort_values(by='name', ascending=False)).head()

(10395, 12)
                                                    ancestor  depth  extinct  \
img_url                                                                        
https://en.wikipedia.org/wiki/File:Wikipedia-lo...      3060   3060     3060   
https://en.wikipedia.org/wiki/File:Stone_Sheep_...        35     35       35   
https://en.wikipedia.org/wiki/File:MoreporkMaun...        22     22       22   
https://en.wikipedia.org/wiki/File:White-bellie...        19     19       19   
https://en.wikipedia.org/wiki/File:Family_Cervi...        19     19       19   
https://en.wikipedia.org/wiki/File:Killdeer_-_C...        15     15       15   
https://en.wikipedia.org/wiki/File:Grass_Owl_ad...        14     14       14   
https://en.wikipedia.org/wiki/File:Aspidoras_fu...        12     12       12   
https://en.wikipedia.org/wiki/File:Xenopus_laev...        12     12       12   
https://en.wikipedia.org/wiki/File:Brown-backed...        11     11       11   
https://en.wikipedia.org/wik

AttributeError: 'NoneType' object has no attribute 'head'

In [12]:
df.to_csv(OUT_FILE, index=False)

In [13]:
#get_image_url("Bill Murray")

In [14]:
#df = pd.read_csv(OUT_FILE)
#df.head()

In [15]:
#print(df.shape)
#df.groupby('img_url').count().sort_values(by='name', ascending=False)

In [16]:
wiki_img_url = df.img_url[0]
wiki_img_url

'https://en.wikipedia.org/wiki/File:image=Kryptopterus.jpg'

In [17]:
def parse_wiki_pic(wiki_img_url, telltale="//upload.wikimedia.org/wikipedia/commons"):
    wiki_pic = requests.get(wiki_img_url).content.decode("utf-8")
    png_str = [b for b in wiki_pic.split('|') if ('.png' in b) and (telltale in b)][0]
    #clean up string
    png_str = re.sub("\\n", '', png_str)
    #png_str = re.sub('^(.+?)href ?= ?\"', '', png_str)
    png_str = re.sub('^(.+?)href ?= ?\"'+telltale, telltale, png_str)
    png_str = re.sub('\"(.+?)$', '', png_str)
    return "https:"+png_str

In [18]:
import base64
import requests
from resizeimage import resizeimage
from PIL import Image
import io

def bytes_to_img(img_bytes, resize=[]):
    image = Image.open(io.BytesIO(img_bytes))
    
    if len(resize) == 2:
        image = resizeimage.resize_contain(image, resize)
    elif len(resize) != 0:
        raise ValueError("resize argument takes a list of length 2 (height x width)")
    
    return image
    
def get_image_base64(url, resize=[]):
    img_bytes = requests.get(url).content
    
    if len(resize) == 2:
        img = bytes_to_img(img_bytes, resize=resize)
        buffered = io.BytesIO()
        img.save(buffered, format="JPEG")
        img_bytes = buffered.getvalue()
    elif len(resize) != 0:
        raise ValueError("resize argument takes a list of length 2 (height x width)")
        
    return base64.b64encode(img_bytes)



In [19]:
#get_image(requests.get(img_url).content, resize=[150,150]).show()

In [20]:
#get_image_base64(img_url, resize=[100,100])

In [21]:
df.img_url[1]

'https://en.wikipedia.org/wiki/File:Castericystis_sprinklei_-_MHNT_-_Millard_County,_Utha,_USA.jpg'

In [22]:
wiki_img_url = df.img_url[1]
real_img_url = parse_wiki_pic(wiki_img_url)
print(real_img_url)
bytes_to_img(requests.get(real_img_url).content, resize=[150,150]).show()
#get_image_base64(real_img_url, resize=[100,100])

https://upload.wikimedia.org/wikipedia/commons/9/95/Castericystis_sprinklei_-_MHNT_-_Millard_County%2C_Utha%2C_USA.jpg


In [23]:
###num_pics = 25
###for i in list(np.random.choice(list(range(df.shape[0])), num_pics)):
###    print(i, end=' ', flush=True)
###    try:
###        wiki_img_url = df.img_url[i]
###        print(df.name[i], end=' ', flush=True)
###        real_img_url = parse_wiki_pic(wiki_img_url)
###        get_image(requests.get(real_img_url).content, resize=[150,150]).show()
###        print("showing "+real_img_url)
###    except:
###        print("-- Bill!")
###        #wiki_img_url = get_image_url("Bill Murray")
###        #real_img_url = parse_wiki_pic(wiki_img_url)
###        #get_image(requests.get(real_img_url).content, resize=[150,150]).show()

In [24]:
wiki_img_url = get_image_url("Bill Murray")
real_img_url = parse_wiki_pic(wiki_img_url)
fallback_bill = get_image_base64(real_img_url, resize=[100,100])

In [ ]:
img_list = []
for i, row in df.iterrows():
    try:
        wiki_img_url = row['img_url']
        real_img_url = parse_wiki_pic(wiki_img_url)
        img_list.append(get_image_base64(real_img_url, resize=[100,100]))
    except:
        #print("-- Bill!")
        img_list.append(fallback_bill)
    if i % 50 == 0:
        print("{}::{}".format(i, int(time()-start)), end= ' ', flush=True)

0::14751 50::14806 100::14844 

/Users/Seth/anaconda/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:785: UserWarning:

Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 



150::14882 200::14932 250::14970 300::15015 350::15041 400::15064 450::15087 500::15144 550::15235 600::15289 650::15357 700::15503 750::15631 800::15686 850::15759 900::15781 950::15839 1000::15967 1050::16063 1100::16088 1150::16134 1200::16182 1250::16322 1300::16413 1350::16505 1400::16627 1450::16681 1500::16760 1550::16838 1600::16868 1650::16941 1700::16991 1750::17051 1800::17214 1850::17287 1900::17424 1950::17558 2000::17690 2050::17761 2100::17830 2150::17957 2200::18132 2250::18219 2300::18310 2350::18469 2400::18614 2450::18712 2500::18822 2550::18895 2600::18970 2650::19056 2700::19164 2750::19224 2800::19267 2850::19343 2900::19409 2950::19461 3000::19522 3050::19584 3100::19638 3150::19700 3200::19760 3250::19800 3300::19854 3350::19906 3400::19969 3450::20023 3500::20085 3550::20182 3600::20258 3650::20307 3700::20356 3750::20422 3800::20459 3850::20535 3900::20617 3950::20671 4000::20736 4050::20808 4100::20871 4150::20931 4200::20996 4250::21065 4300::21144 4350::212

In [ ]:
df['img_base64'] = img_list
df.tail()

In [ ]:
df.to_csv(OUT_FILE.replace(".csv", "-base64.csv"), index=False)